# Set api key

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# create vectordb

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    collection_name="collection2",
    embedding_function=embeddings,
    persist_directory="./chrome_vector_db",
)

In [3]:
from pydantic import BaseModel, Field
from typing import Optional

class Attributes(BaseModel):
    length: Optional[float] = Field(default=None, description="length in inches. if the name uses different units, convert to inches")
    gauge: Optional[int] = Field(default=None, description="gauge in inches. for example, 20ga or 20g refers to 20 gauge") 

In [4]:
import pandas as pd

catalog_df = pd.read_csv("documents/ProductDB_clean.csv")

In [5]:
catalog_df

,PRODUCT_ID,PRODUCT_NO,NAME,ALIAS,STATUS,PRODUCT_LINE_CODE,PRODUCT_GROUP_CODE,PRODUCT_CODE_CODE,Product,SFIA
0,4802,1011,18MIL 1-5/8IN DRYWALL STUD,162S125-18,1,FRAM,DWF,DWS,1011,162S125-18
1,4804,1013,DRYWALL STUD 1-5/8IN 20G,162S125-30 (20G),1,FRAM,DWF,DWS,1013,162S125-30 (20G)
2,4806,1021,18MIL 2-1/2IN DRYWALL STUD,250S125-18,1,FRAM,DWF,DWS,1021,250S125-18
3,4808,1023,DRYWALL STUD 2-1/2IN 20G,250S125-30 (20G),1,FRAM,DWF,DWS,1023,250S125-30 (20G)
4,4811,1031,18MIL 3-5/8IN DRYWALL STUD,362S125-18,1,FRAM,DWF,DWS,1031,362S125-18
...,...,...,...,...,...,...,...,...,...,...
9808,9099,99942,ANGL 1.5 X 2 (14GA),NaN,1,FRAM,ACC,ANGL,99942,150L200-68
9809,9102,99947,WBS 400 X 13.75 IN-14,NaN,1,CONN,BRAC,WBS,99947,400WBS-68
9810,35945,999995,Price Protection,NaN,1,MISC,MISC,MISC,999995,Price Protection
9811,11808,999998,Generic Part,NaN,1,FRAM,ACC,ANGL,999998,Generic Part


In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)
structured_llm = model.with_structured_output(Attributes)

In [7]:
from langchain_core.prompts.chat import ChatPromptTemplate
get_attr_prompt_template = ChatPromptTemplate.from_template("Parse all given attributes from the given product name. If the attribute isn't present in the name, or you are unsure, then leave it as None. If a measurement is written like 2-1/2IN, then the measurement is 2.5 in. \nProduct Name: {product_name}")

In [8]:
get_attr_prompt = get_attr_prompt_template.invoke({'product_name': 'DRYWALL STUD 1-5/8IN 20G'})
structured_llm.invoke(get_attr_prompt)

Attributes(length=1.625, gauge=20)

In [9]:
from langchain_core.documents import Document
for index, row in catalog_df.iterrows():
    product_name = str(row["NAME"])
    alias = str(row["ALIAS"])
    name = product_name + " " + alias
    get_attr_prompt = get_attr_prompt_template.invoke({'product_name': name})
    parsed_attributes = structured_llm.invoke(get_attr_prompt)
    attr_dict = dict(parsed_attributes)
    metadata = {key:val for key, val in attr_dict.items() if val is not None}
    metadata['PRODUCT_NO'] = row["PRODUCT_NO"]
    document = Document(page_content=name, metadata=metadata)
    vector_store.add_documents([document])
    print(f"Added document {index+1}/{len(catalog_df)} with name {name} and attributes length {attr_dict['length']} and gauge {attr_dict['gauge']}")

Added document 1/9813 with name 18MIL 1-5/8IN DRYWALL STUD 162S125-18 and attributes length 1.625 and gauge 18
Added document 2/9813 with name DRYWALL STUD 1-5/8IN 20G 162S125-30 (20G) and attributes length 1.625 and gauge 20
Added document 3/9813 with name 18MIL 2-1/2IN DRYWALL STUD 250S125-18 and attributes length 2.5 and gauge 18
Added document 4/9813 with name DRYWALL STUD 2-1/2IN 20G 250S125-30 (20G) and attributes length 2.5 and gauge 20
Added document 5/9813 with name 18MIL 3-5/8IN DRYWALL STUD 362S125-18 and attributes length 3.625 and gauge 18
Added document 6/9813 with name 18MIL 4IN DRYWALL STUD 400S125-18 and attributes length 4.0 and gauge 18
Added document 7/9813 with name DRYWALL STUD 4IN 20G 400S125-30 (20G) and attributes length 4.0 and gauge 20
Added document 8/9813 with name 18MIL 6IN DRYWALL STUD 600S125-18 and attributes length 6.0 and gauge 18
Added document 9/9813 with name 10FT GALVANIZED BULLNOSE CORNER BEAD nan and attributes length 120.0 and gauge None
Added 

KeyboardInterrupt: 